In [ ]:
import pandas as pd
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

data_basepath = '..\\data\\spain_dataframes\\'

In [ ]:
# Leer el fichero con los datos
df = pd.read_csv(f'{data_basepath}datos_tratados.csv')
df

### Homicidios y Criminalidad --> Percepción de seguridad

Se establece una nota media de la percepción de la seguridad para cada individuo, en función de la media de las notas de los homicidios y la criminalidad.

In [ ]:
# Valoración de una columna dentro de un rango determinado
def valorate_numeric_column(df, column_name, new_column_name, min_rank_value=0, max_rank_value=1, revert_rank_valoration=False):
    """
    Normaliza los valores de una columna de un DataFrame dentro de un rango específico y crea una nueva columna con los resultados.

    La valoración se realiza de forma lineal, permitiendo definir un rango mínimo y máximo.
    Se toman como valores extremos los valores mínimo y máximo de la columna especificada en 'column_name'.

    Parámetros:
    -----------
    df : pandas.DataFrame
        DataFrame que contiene la columna a normalizar.
    column_name : str
        Nombre de la columna que se va a normalizar.
    new_column_name : str
        Nombre de la nueva columna donde se almacenarán los valores normalizados.
    min_rank_value : float, opcional (por defecto=0)
        Valor mínimo del rango.
    max_rank_value : float, opcional (por defecto=1)
        Valor máximo del rango.
    revert_rank_valoration : bool, opcional (por defecto=False)
        Si es True, invierte la valoración (el valor más alto se convierte en el más bajo y viceversa).

    Retorna:
    --------
    None. Modifica el DataFrame original añadiendo una nueva columna con los valores normalizados.
    """
    # Valores extremos del dataframe
    min_val = df[column_name].min()
    max_val = df[column_name].max()

    # Proporcionalidad entre el rango definido y el rango calculado
    rank_diference = max_rank_value - min_rank_value

    # Aplicar la fórmula de normalización lineal
    if not revert_rank_valoration:
        df[new_column_name] = ((df[column_name] - min_val) / (max_val - min_val)) * rank_diference + min_rank_value
    else:
        df[new_column_name] = (1 - ((df[column_name] - min_val) / (max_val - min_val))) * rank_diference + min_rank_value

In [ ]:
# Valoración de la criminalidad
valorate_numeric_column(df, 'criminalidad_1000habit', 'valoracion_criminalidad', 0, 10, True)

# Valoración de la tasa de homicidios
valorate_numeric_column(df, 'homicidios_100mhabit', 'valoracion_homicidios', 0, 10, True)

df[['criminalidad_1000habit', 'valoracion_criminalidad', 'homicidios_100mhabit', 'valoracion_homicidios']].head()

In [ ]:
# Hacer una valoración media "percepcion_seguridad" entre la criminalidad y la tasa de homicidios
df['percepcion_seguridad'] = (df['valoracion_criminalidad'] + df['valoracion_homicidios']) / 2
df['percepcion_seguridad']

### Estado civil

Se limpian los valores de estado civil, siendo soltero/a para los menores de 18 años, y el resto de valores se mantienen.

In [ ]:
df.loc[df['edad'] < 18, 'estado_civil'] = 'soltero/a'
df[df['edad'] < 18]['estado_civil'].unique()

### Nivel de estudios

Se limpian los valores de estudios que por edad no son coherentes con el máximo nivel de estudios alcanzado.

In [ ]:
df['estudios'].unique().tolist()

In [ ]:
# Se ordenan los niveles de educación de mayor a menor nivel
ordered_education = ['educacion_superior',
                     'segundo_secundaria_profesional_',
                     'segundo_secundaria_general',
                     'primero_secundaria',
                     'primaria',
                     'estudios_primarios_incompletos',
                     'analfabetos']

In [ ]:
# Se recorre todo el dataframe para comprobar registro por registro el nivel de estudios alcanzado por cada persona en función de su edad.
iterator = 0 # Iterador entre profesional y general en secundaria
for index, row in df.iterrows():
    # Comprobamos el nivel de estudios máximo al que puede obtar por su edad

    if row['edad'] < 10 and ordered_education.index(row['estudios']) < 5:
        # Si es menor de 10, los máximos estudios alcanzados son estudios primarios incompletos
        df.loc[index, 'estudios'] = ordered_education[5]

    if row['edad'] < 12 and ordered_education.index(row['estudios']) < 4:
        # Si es menor de 12, los máximos estudios alcanzados son primaria
        df.loc[index, 'estudios'] = ordered_education[4]

    elif row['edad'] < 16 and ordered_education.index(row['estudios']) < 3:
        # Si es menor de 16, los máximos estudios alcanzados son primero de secundaria
        df.loc[index, 'estudios'] = ordered_education[3]

    elif row['edad'] < 18 and ordered_education.index(row['estudios']) < 2:
        # Si es menor de 18, los máximos estudios alcanzados son segundo de secundaria
        df.loc[index, 'estudios'] = ordered_education[1 + iterator]
        iterator = 1 if iterator == 0 else 0 # Cambia entre profesional y general para la siguiente iteración

In [ ]:
df[df['edad'] < 23]['estudios'].unique()

### Salario anual

Se establece un salario 0 para menores de 16 años.

In [ ]:
df.loc[df['edad'] < 16, 'salario_anual'] = 0

### Horas trabajadas al mes

Se establece un valor de 0 horas trabajadas al mes para menores de 16 años.

In [ ]:
df.loc[df['edad'] < 16, 'horasTrabajadas_mes'] = 0

### Actividad física

Se establece una actividad física `nivel_bajo` para mayores de 75 años.

In [ ]:
df.loc[df['edad'] > 75, 'actividad_fisica'] = 'nivel_bajo'

### Adecuación de columnas

Se eliminará información redundante de algunas columnas.

In [ ]:
df

In [ ]:
# Eliminar el texto "hospitales_" de los registros tipo string en todas las columnas
df = df.applymap(lambda x: x.replace('hospitales_', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace('dentistas_', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace('especialistas_', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace('medGeneral_', '') if isinstance(x, str) else x)

### Eliminación de columnas

Se eliminan columnas innecesarias para el análisis.

In [ ]:
df.drop(columns=['valoracion_criminalidad'], inplace=True)
df.drop(columns=['valoracion_homicidios'], inplace=True)
df.drop(columns=['homicidios_100mhabit'], inplace=True)
df.drop(columns=['criminalidad_1000habit'], inplace=True)

In [ ]:
df

## Guardado de los datos

In [ ]:
# Definir la ruta completa del archivo CSV
filename = 'cleaned_data.csv'
file_path = f'{data_basepath}..\\spain_dataframes\\{filename}'

# Guardar el DataFrame en un archivo CSV
df.to_csv(file_path, index=False)

print(f"DataFrame guardado exitosamente en '{filename}'")